In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [2]:
# Define the path to your dataset, where each class has its own folder.
train_data_dir = './newData/'
valid_data_dir = './newData'

# Define image size and batch size.
img_size = (256,256)
batch_size = 8

# Create data generators for training and validation data.
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.3,
    rotation_range=15,
    width_shift_range=0.2, # horizontal shift
    height_shift_range=0.2,
    validation_split=0.2  # Split data into training and validation.
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'  # Specify 'training' for the training generator.
)

valid_datagen = ImageDataGenerator(
    rescale=1.0/255
)
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify 'validation' for the validation generator.
)

Found 22260 images belonging to 29 classes.
Found 5554 images belonging to 29 classes.


In [3]:
class_names = list(train_generator.class_indices.keys())

In [4]:
n_classes = len(class_names)
# Define the CNN model.
model = keras.Sequential([
    keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(256,256, 3)),
    keras.layers.Conv2D(16, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(256, (3, 3), activation='relu'),
    keras.layers.Conv2D(256, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(n_classes, activation='softmax') 
])


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 16)      2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 126, 126, 16)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 32)      4640      
                                                                 
 conv2d_3 (Conv2D)           (None, 122, 122, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 61, 61, 32)       0         
 2D)                                                    

In [6]:
model.compile(
    optimizer= tf.keras.optimizers.experimental.Adam(learning_rate=0.00018),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [7]:
# Train the model.
history = model.fit(
    train_generator,
    epochs=35,  # You can adjust the number of epochs.
    validation_data=valid_generator,
    callbacks=[EarlyStopping(patience=20),ReduceLROnPlateau(monitor='val_loss', factor=0.3,patience=2, min_lr=0.000000001)]
)

Epoch 1/35
2783/2783 [==============================] - 522s 182ms/step - loss: 2.7457 - accuracy: 0.1654 - val_loss: 2.3714 - val_accuracy: 0.2488 - lr: 1.8000e-04
Epoch 2/35
2783/2783 [==============================] - 254s 91ms/step - loss: 1.6135 - accuracy: 0.4831 - val_loss: 1.3073 - val_accuracy: 0.5821 - lr: 1.8000e-04
Epoch 3/35
2783/2783 [==============================] - 247s 89ms/step - loss: 1.0201 - accuracy: 0.6711 - val_loss: 0.8132 - val_accuracy: 0.7299 - lr: 1.8000e-04
Epoch 4/35
2783/2783 [==============================] - 246s 88ms/step - loss: 0.7297 - accuracy: 0.7602 - val_loss: 0.5828 - val_accuracy: 0.8059 - lr: 1.8000e-04
Epoch 5/35
2783/2783 [==============================] - 248s 89ms/step - loss: 0.5774 - accuracy: 0.8115 - val_loss: 0.5245 - val_accuracy: 0.8239 - lr: 1.8000e-04
Epoch 6/35
2783/2783 [==============================] - 248s 89ms/step - loss: 0.4762 - accuracy: 0.8481 - val_loss: 0.5145 - val_accuracy: 0.8268 - lr: 1.8000e-04
Epoch 7/35
2783

In [8]:
model.save("./model files/PlantDiseaseDetection.h5")